In [1]:
import pandas as pd

df = pd.read_csv('./malpedia.csv')
# df = pd.DataFrame(report_json_list)
# df.to_csv('./malpedia.csv', index=False)
df.head(3)

,media_type,url,date,organization,title,author
0,online,https://github.com/649/APT38-DYEPACK,2019-03-20,Github (649),APT38 DYEPACK FRAMEWORK,@037
1,online,https://blueteamblog.com/microsoft-exchange-ze...,2021-03-06,Blue Team Blog,Microsoft Exchange Zero Day’s – Mitigations an...,Auth 0r
2,online,https://blueteamblog.com/darkside-ransomware-o...,2021-05-14,Blue Team Blog,DarkSide Ransomware Operations – Preventions a...,Auth 0r


介於 4 ~ 237，平均 54。那檔名總長度取60，多的喀調。。

In [11]:
df['len'] = df.apply(lambda row: len(row.title), axis = 1)
max = df['len'].max()
min = df['len'].min()
mean = df['len'].mean()
print(max, min, mean)

237 4 54.90650640564272


處理重複的檔名
- The Eye of the Tiger 
- The Eye of the Tiger-no2
- The Eye of the Tiger-no3

In [40]:
import unicodedata
import re

def slugify(value, allow_unicode=False):
    """
    https://stackoverflow.com/questions/295135/turn-a-string-into-a-valid-filename
    Taken from https://github.com/django/django/blob/master/django/utils/text.py
    Convert to ASCII if 'allow_unicode' is False. Convert spaces or repeated
    dashes to single dashes. Remove characters that aren't alphanumerics,
    underscores, or hyphens. Convert to lowercase. Also strip leading and
    trailing whitespace, dashes, and underscores.
    """
    value = str(value)
    if allow_unicode:
        value = unicodedata.normalize('NFKC', value)
    else:
        value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore').decode('ascii')
    value = re.sub(r'[^\w\s-]', '', value) # , value.lower()
    return re.sub(r'[-\s]+', '-', value).strip('-_')

slugify("Thwarting Loaders: From SocGholish to BLISTER’s LockBit Payl")

'Thwarting-Loaders-From-SocGholish-to-BLISTERs-LockBit-Payl'

In [60]:
df = pd.read_csv('./malpedia.csv')
df['filename'] = df.apply(lambda row: slugify(row.title)[:60], axis = 1)    # max len 60
df['filename'] = df.apply(lambda row: 'no-title' if row.filename == '' else row.filename, axis = 1)

In [61]:
# view duplicates
from collections import Counter
counter = Counter(df['filename'])
counter.most_common(20)

[('no-title', 7),
 ('The-Eye-of-the-Tiger', 4),
 ('Tweet-on-Malware-Sample', 4),
 ('Operation-Cleaver', 4),
 ('Hidden-Lynx-Professional-Hackers-for-Hire', 4),
 ('Lazarus-under-the-Hood', 4),
 ('Analysis-Run', 4),
 ('The-Gamaredon-Group-Toolset-Evolution', 4),
 ('Purple-Fox-Uses-New-Arrival-Vector-and-Improves-Malware-Arse', 4),
 ('Buckeye-cyberespionage-group-shifts-gaze-from-US-to-Hong-Kon', 4),
 ('APT', 3),
 ('Axiom', 3),
 ('APT38', 3),
 ('Prince-of-Persia-Infy-Malware-Active-In-Decade-of-Targeted-A', 3),
 ('Pacifier-APT', 3),
 ('Spoofing-in-the-reeds-with-Rietspoof', 3),
 ('Masters-of-Mimicry-new-APT-group-ChamelGang-and-its-arsenal', 3),
 ('PseudoManuscrypt-a-mass-scale-spyware-attack-campaign', 3),
 ('China-Hacked-South-Korea-Over-Missile-Defense-US-Firm-Says', 3),
 ('Thwarting-Loaders-From-SocGholish-to-BLISTERs-LockBit-Payloa', 3)]

In [62]:
# address duplicates
current_cnt = dict.fromkeys(counter.keys(), 0)
for i, row in df.iterrows():
    name = row['filename']
    current_cnt[name] += 1
    if current_cnt[name] >= 2:
        name = f"{name}-no{current_cnt[name]}"
    df.at[i,'filename'] = name

from collections import Counter
counter = Counter(df['filename'])
counter.most_common(3)

[('APT38-DYEPACK-FRAMEWORK', 1),
 ('Microsoft-Exchange-Zero-Days-Mitigations-and-Detections', 1),
 ('DarkSide-Ransomware-Operations-Preventions-and-Detections', 1)]

In [63]:
# add column 表示尚未下載
df['download'] = 0

# save new csv file
df.to_csv('./malpedia.csv', index=False)

In [64]:
len(df['filename'])

13894

In [65]:
len(set(df['filename']))

13894